# Memory

In [chapter 3](https://learn.deeplearning.ai/courses/langchain/lesson/3/memory) the topic was Memory, i.e. how to make LLMs remember information. LLMs are inherently stateless, i.e. they don't remember information from one call to the next. Therefore, we need to pass the previous conversation as context to the LLM to create a chat-like experience.

In my previous blog post on [building chat from scratch](https://chrwittm.github.io/posts/2024-02-23-chat-from-scratch/), I implemented a chat messages class to store the conversation history. A chat client class handled the conversation to the LLM.

Following the lesson, let's first re-create the chat experience using langchain.

Afterwards, I will implement a use case including conversation summarization. Even though, the context sizes have increased massively over the past year, and cost per token has significantly declined, I still find the approach interesting, and it should be a good learning exercise.

## Chat with langchain

Let's start by importing the necessary modules and initializing the LLM and memory.

In [117]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain
from langchain.memory import ConversationBufferMemory
from langchain.schema import SystemMessage

# Initialize the LLM
llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

# Initialize the memory
memory = ConversationBufferMemory()

# Add system message to the memory
system_message = SystemMessage(content="You are a helpful assistant. Please provide concise answers.")
memory.chat_memory.add_message(system_message)

# Initialize the conversation
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False # Set to True to see the prompt and response
)

In [118]:
# add system prompt


#prompt = "Hi, my name is Christian"
prompt = "Please list the planets in the solar system"
model_response = conversation.predict(input=prompt)
print(memory.buffer)

System: You are a helpful assistant. Please provide concise answers.
Human: Please list the planets in the solar system
AI: Sure! The planets in our solar system, in order from the Sun, are:

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

There are also dwarf planets, like Pluto, but those are not classified as the main planets. Let me know if you want more information about any of them!


In [119]:
prompt = "Please reverse the list"
model_response = conversation.predict(input=prompt)
print(memory.buffer)



System: You are a helpful assistant. Please provide concise answers.
Human: Please list the planets in the solar system
AI: Sure! The planets in our solar system, in order from the Sun, are:

1. Mercury
2. Venus
3. Earth
4. Mars
5. Jupiter
6. Saturn
7. Uranus
8. Neptune

There are also dwarf planets, like Pluto, but those are not classified as the main planets. Let me know if you want more information about any of them!
Human: Please reverse the list
AI: Of course! Here’s the list of planets in reverse order:

1. Neptune
2. Uranus
3. Saturn
4. Jupiter
5. Mars
6. Earth
7. Venus
8. Mercury

If you need anything else, feel free to ask!


As we have learned in the lesson, we can also inject previous messages / a conversation history into the LLM prompt.

In [120]:
# Re-Initialize the memory
memory = ConversationBufferMemory()

conversation.memory = memory

memory.save_context({"input": "Hi"}, 
                    {"output": "What's up"})



In [121]:
print(memory.buffer)

Human: Hi
AI: What's up


In [122]:
prompt = "Not much, just hanging"
model_response = conversation.predict(input=prompt)
print(memory.buffer)

Human: Hi
AI: What's up
Human: Not much, just hanging
AI: That sounds nice! Sometimes just taking it easy is the best way to spend your time. Do you have any plans for the day, or are you just going with the flow?


## Conversation Summarization

The lesson introduced different types of memory. I am not going to repeat all the examples, but I will show how to implement the conversation summarization use case.

For conversation summarization, we will use the [Daily Dialog](https://huggingface.co/datasets/li2017dailydialog/daily_dialog) dataset.

In [123]:
from datasets import load_dataset
import os
from dotenv import load_dotenv

# Load the .env file
load_dotenv()

# Get the token from the environment variable
hf_token = os.getenv("HF_TOKEN")


In [124]:
# Authenticate with Hugging Face Hub
from huggingface_hub import login
login(token=hf_token)

# Load the dataset
dataset = load_dataset('daily_dialog')

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: read).
Your token has been saved to /Users/chrwittm/.cache/huggingface/token
Login successful


In [125]:
dataset

DatasetDict({
    train: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 11118
    })
    validation: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialog', 'act', 'emotion'],
        num_rows: 1000
    })
})

The dataset contains more than 10k conversations. 

Let's take a look at one conversation.


In [126]:
index = 0
dialog = dataset['train'][index]['dialog']
dialog

['Say , Jim , how about going for a few beers after dinner ? ',
 ' You know that is tempting but is really not good for our fitness . ',
 ' What do you mean ? It will help us to relax . ',
 " Do you really think so ? I don't . It will just make us fat and act silly . Remember last time ? ",
 " I guess you are right.But what shall we do ? I don't feel like sitting at home . ",
 ' I suggest a walk over to the gym where we can play singsong and meet some of our friends . ',
 " That's a good idea . I hear Mary and Sally often go there to play pingpong.Perhaps we can make a foursome with them . ",
 ' Sounds great to me ! If they are willing , we could ask them to go dancing with us.That is excellent exercise and fun , too . ',
 " Good.Let ' s go now . ",
 ' All right . ']

More let's move the dialog into the langchain memory.










In [127]:
from langchain.memory import ConversationSummaryBufferMemory

Let's add the conversation to the memory and print the buffer to see what is stored.

In [128]:
memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=50)

for i in range(0, len(dialog),2):
    memory.save_context({"input": dialog[i]}, {"output": dialog[i+1]})

print(memory.buffer)



System: The human suggests going for a few beers after dinner, but the AI declines, stating that it is tempting but not good for their fitness. The human argues that it would help them relax, but the AI disagrees, recalling that it would just lead to negative consequences like weight gain and silliness, referencing a previous experience. The human concedes the AI's point but expresses a desire to do something other than stay at home. The AI then suggests a walk to the gym to play singsong and meet friends, to which the human agrees, mentioning that Mary and Sally often go there to play ping pong and suggesting they could make a foursome with them. The AI responds positively, suggesting that if Mary and Sally are willing, they could also ask them to go dancing, which the AI considers excellent exercise and fun.
Human:  Good.Let ' s go now . 
AI:  All right . 


As we can see, the stores the summarized conversation. Let's continue the conversation.

In [130]:
# Initialize the conversation with the memory
conversation = ConversationChain(
    llm=llm, 
    memory = memory,
    verbose=False
)

prompt = "I am glad you came up with this idea"
model_response = conversation.predict(input=prompt)
print(model_response)

I'm really glad you like it! It's always nice to mix a bit of exercise with socializing. Plus, the fresh air will do us good! Do you have any favorite songs in mind for our singsong session? I think it would be fun to belt out some classics or maybe even some upbeat tunes to get us in the mood for dancing later!


The summary of the conversation has been updated.

In [110]:
print(memory.buffer)

System: The human suggests going for a few beers after dinner, but the AI declines, stating that it is tempting but not good for their fitness. The human argues that it would help them relax, but the AI disagrees, recalling that it would lead to negative consequences like weight gain and silly behavior. The human concedes but expresses a desire to do something other than stay at home. The AI then suggests a walk to the gym to play singsong and meet friends, to which the human agrees, mentioning that Mary and Sally often go there to play ping pong and suggesting they could make a foursome with them. The AI proposes that if Mary and Sally are willing, they could also ask them to go dancing, which the AI considers excellent exercise and fun. The human agrees to go now, and the AI expresses happiness that the human liked the idea, emphasizing the benefits of fresh air and catching up with friends. The AI also asks if the human has a favorite song to sing during singsong, suggesting it woul